## 128. 最长连续序列
给定一个未排序的整数数组，找出最长连续序列的长度。

要求算法的时间复杂度为 O(n)。

示例:

输入: [100, 4, 200, 1, 3, 2] 输出: 4 解释: 最长连续序列是 [1, 2, 3, 4]。它的长度为 4。

In [ ]:
class Solution {
public:
    int longestConsecutive(vector<int>& nums) {
        //时间复杂度O(n)，空间复杂度O(n)
        int res=0;
        unordered_set<int> m(nums.begin(),nums.end());  //建立一个集合
        for(int  a : nums){
            if(!m.count(a))     
                continue;    //跳过被删除的元素
            int left=a-1,right=a+1;
            while(m.count(left))      //遍历nums ,  对其中每一个元素遍历左右序列  
                m.erase(left--);     //  遍历一次后删除这个元素，避免重复计算
            while(m.count(right))
                m.erase(right++);
            res=max(res,right-left-1);  //保存最大序列长度
        }
        return res;
        
    }
};

## 146. LRU缓存机制
运用你所掌握的数据结构，设计和实现一个 LRU (最近最少使用) 缓存机制。它应该支持以下操作： 获取数据 get 和 写入数据 put 。

获取数据 get(key) - 如果密钥 (key) 存在于缓存中，则获取密钥的值（总是正数），否则返回 -1。 写入数据 put(key, value) - 如果密钥不存在，则写入其数据值。当缓存容量达到上限时，它应该在写入新数据之前删除最近最少使用的数据值，从而为新的数据值留出空间。

进阶:

你是否可以在 O(1) 时间复杂度内完成这两种操作？

示例:

LRUCache cache = new LRUCache( 2 /* 缓存容量 */ );

cache.put(1, 1); cache.put(2, 2); cache.get(1); // 返回 1 cache.put(3, 3); // 该操作会使得密钥 2 作废 cache.get(2); // 返回 -1 (未找到) cache.put(4, 4); // 该操作会使得密钥 1 作废 cache.get(1); // 返回 -1 (未找到) cache.get(3); // 返回 3 cache.get(4); // 返回 4

我们需要三个私有变量，nums, l和m，其中 nums 是缓存器的容量大小，l是保存缓存器内容的列表，m是 HashMap，保存关键值 key 和缓存器各项的迭代器之间映射，方便我们以 O(1) 的时间内找到目标项。

- get:我们在 HashMap 中查找给定的 key，若不存在直接返回 -1。如果存在则将此项移到顶部，这里我们使用 C++ STL 中的函数 splice，专门移动链表中的一个或若干个结点到某个特定的位置，这里我们就只移动 key 对应的迭代器到列表的开头，然后返回 value。
- put:我们也是现在 HashMap 中查找给定的 key，如果存在就删掉原有项，并在顶部插入新来项，然后判断是否溢出，若溢出则删掉底部项(最不常用项)。

In [ ]:
class LRUCache {
public:
    LRUCache(int capacity) {
        nums=capacity;
    }
    
    int get(int key) {
        auto it=m.find(key);
        if(it==m.end())
            return -1;
        l.splice(l.begin(),l,it->second);
        return it->second->second;
    }
    
    void put(int key, int value) {
        auto it=m.find(key);
        if(it!=m.end())
            l.erase(it->second);
        l.push_front(make_pair(key,value));
        m[key]=l.begin();
        if(m.size()>nums){
            int k=l.rbegin()->first;
            l.pop_back();
            m.erase(k);
        }
    }
private:
    int nums;
    list<pair<int,int>> l;
    map<int,list<pair<int,int>>::iterator> m;
    
};

## 380. 常数时间插入、删除和获取随机元素
设计一个支持在平均 时间复杂度 O(1) 下，执行以下操作的数据结构。

insert(val)：当元素 val 不存在时，向集合中插入该项。 remove(val)：元素 val 存在时，从集合中移除该项。 getRandom：随机返回现有集合中的一项。每个元素应该有相同的概率被返回。 示例 :

// 初始化一个空的集合。 RandomizedSet randomSet = new RandomizedSet();

// 向集合中插入 1 。返回 true 表示 1 被成功地插入。 randomSet.insert(1);

// 返回 false ，表示集合中不存在 2 。 randomSet.remove(2);

// 向集合中插入 2 。返回 true 。集合现在包含 [1,2] 。 randomSet.insert(2);

// getRandom 应随机返回 1 或 2 。 randomSet.getRandom();

// 从集合中移除 1 ，返回 true 。集合现在包含 [2] 。 randomSet.remove(1);

// 2 已在集合中，所以返回 false 。 randomSet.insert(2);

// 由于 2 是集合中唯一的数字，getRandom 总是返回 2 。 randomSet.getRandom();

此题的正确解法是利用到了一个一维数组和一个 HashMap，其中数组用来保存数字，HashMap 用来建立每个数字和其在数组中的位置之间的映射，

- 对于插入操作，我们先看这个数字是否已经在 HashMap 中存在，如果存在的话直接返回 false，不存在的话，我们将其插入到数组的末尾，然后建立数字和其位置的映射。
- 删除操作是比较 tricky 的，我们还是要先判断其是否在 HashMap 里，如果没有，直接返回 false。由于 HashMap 的删除是常数时间的，而数组并不是，为了使数组删除也能常数级，我们实际上将要删除的数字和数组的最后一个数字调换个位置，然后修改对应的 HashMap 中的值，这样我们只需要删除数组的最后一个元素即可，保证了常数时间内的删除。
- 返回随机数对于数组来说就很简单了，我们只要随机生成一个位置，返回该位置上的数字即可

In [ ]:
class RandomizedSet {
public:
    /** Initialize your data structure here. */
    RandomizedSet() {
        
    }
    
    /** Inserts a value to the set. Returns true if the set did not already contain the specified element. */
    bool insert(int val) {
        if(m.count(val)){
            return false;
        }
        nums.push_back(val);
        m[val]=nums.size()-1;
        return true;
    }
    
    /** Removes a value from the set. Returns true if the set contained the specified element. */
    bool remove(int val) {
        if(!m.count(val))
            return false;
        int last=nums.back();
        m[last]=m[val];        //修改对应的hashmap中的位置值
        nums[m[val]]=last;
        nums.pop_back();
        m.erase(val);
        return true;
    }
    
    /** Get a random element from the set. */
    int getRandom() {
        return nums[rand()%nums.size()];
    }
private:
    vector<int> nums;
    unordered_map<int,int> m;
};

/**
 * Your RandomizedSet object will be instantiated and called as such:
 * RandomizedSet* obj = new RandomizedSet();
 * bool param_1 = obj->insert(val);
 * bool param_2 = obj->remove(val);
 * int param_3 = obj->getRandom();
 */

## 715. Range 模块
Range 模块是跟踪数字范围的模块。你的任务是以一种有效的方式设计和实现以下接口。

- addRange(int left, int right) 添加半开区间 [left, right)，跟踪该区间中的每个实数。添加与当前跟踪的数字部分重叠的区间时，应当添加在区间 [left, right) 中尚未跟踪的任何数字到该区间中。
- queryRange(int left, int right) 只有在当前正在跟踪区间 [left, right) 中的每一个实数时，才返回 true。
- removeRange(int left, int right) 停止跟踪区间 [left, right) 中当前正在跟踪的每个实数。
 

示例：

- addRange(10, 20): null
- removeRange(14, 16): null
- queryRange(10, 14): true （区间[10, 14) 中的每个数都正在被跟踪）
- queryRange(13, 15): false （未跟踪区间 [13, 15) 中像 14, 14.03, 14.17 这样的数字）
- queryRange(16, 17): true （尽管执行了删除操作，区间 [16, 17) 中的数字 16 仍然会被跟踪）


In [ ]:
class RangeModule {
public:
    RangeModule() { }
    void addRange(int left, int right) {
        vector<pair<int ,int>> res;
        int n=v.size(),cur=0;
        for(int i=0;i<n;++i){
            if(v[i].second<left){
                res.push_back(v[i]); //将所有右边界小于left的区间存入res
                ++cur;   //  记录右边界中仅次于left的位置，也是区间【left,right)插入位置
            }
            else if(v[i].first>right){
                res.push_back(v[i]);
            }
            else{
                left=min(left,v[i].first);
                right=max(right,v[i].second);
            }
        }
        res.insert(res.begin()+cur,{left,right});
        v=res;
    }
    
    bool queryRange(int left, int right) {
        for(auto q:v){
            if(q.first<=left&&q.second>=right)
                return true;
        }
        return false;
    }
    
    void removeRange(int left, int right) {
        vector<pair<int ,int>> res,t;
        int n=v.size(),cur=0;
        for(int i=0;i<n;++i){
            if(v[i].second<=left){
                res.push_back(v[i]);
                ++cur;
            }
            else if(v[i].first>=right){
                res.push_back(v[i]);
            }
            else{
                if(v[i].first<left){
                    t.push_back({v[i].first,left});
                }
                if(v[i].second>right){
                    t.push_back({right,v[i].second});
                }
            }
        }
        res.insert(res.begin()+cur,t.begin(),t.end());
        v=res;
    }
    private:
    vector<pair<int ,int>> v;
};

/**
 * Your RangeModule object will be instantiated and called as such:
 * RangeModule* obj = new RangeModule();
 * obj->addRange(left,right);
 * bool param_2 = obj->queryRange(left,right);
 * obj->removeRange(left,right);
 */